In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import talib
from sklearn.decomposition import PCA
import math
import plotly.express as px
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split



In [2]:

PATH = "../Data/DAX/yahoo_fin/"

def load(filename="1d"):
     data = pd.read_csv(PATH+"{}.csv".format(filename))
     return data
def save(filename, data):
     data.to_csv(PATH+filename+".csv")

def normalize(x):
    mean = np.mean(x, axis=0, keepdims=True)
    std = np.std(x, axis=0, keepdims=True)
    return (x - mean) / std


In [51]:
data = load("1997-01-01-2020-08-04:1d")
data2 = load("1997-01-01-2020-08-04:1wk")
data3 = load("1997-01-01-2020-08-04:1mo")

data_all = data.append(data2, ignore_index=True)
data_all = data.append(data3, ignore_index=True)
data2 = data2.append(data3, ignore_index=True)

# print(data.columns)
# print(data.shape)


In [40]:
def tf_model(period):
    feature_list = ["1-rad", "inter-derivative", 
                    "bb-upper", "bb-lower", "bb-middle", 
                    "ema-cross", "sma-cross", 
                    "macdsignal", "macd", "macdhist",
                    "rsi", "sar-diff", 
                    "intra-derivative", "1-err", "intra-diff"]
    
    N = len(feature_list)
    features = period[feature_list][:-1]
    features = np.nan_to_num(features.values)
    features = normalize(features)

    targets = period["inter-diff"][1:]
    targets = targets.values

    train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size = 0.20, random_state = 2020)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(N),
        tf.keras.layers.Dense(128*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(N, activation='relu'),
        # tf.keras.layers.Dense(N/2, activation='relu'),
        tf.keras.layers.Dense(1)
        ])

    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


    model.fit(train_features, train_labels, epochs=500, batch_size=100)
    model.evaluate(test_features,  test_labels, verbose=2)
    model.predict(test_features)
    return model

def tf_model2(period):
    feature_list = ["1-rad", "inter-derivative", 
                    "bb-upper", "bb-lower", "bb-middle", 
                    "ema-cross", "sma-cross", 
                    "macdsignal", "macd", "macdhist",
                    "rsi", "sar-diff", 
                    "intra-derivative", "1-err", "intra-diff"]

    N = len(feature_list)
    features = period[feature_list][:-1]
    features = np.nan_to_num(features.values)
    features = normalize(features)

    targets = period["inter-diff"][1:]
    targets = targets.values

    train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size = 0.20, random_state = 2020)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(N),
        tf.keras.layers.Dense(128*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1)
        ])

    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


    model.fit(train_features, train_labels, epochs=500, batch_size=100, verbose=0)
    model.evaluate(test_features,  test_labels, verbose=2)
    model.predict(test_features)
    return model
    

In [48]:
m1 = tf_model(data2)

# m2 = tf_model2(data)


 0.5122
Epoch 323/500
13/13 [==============================] - 0s 2ms/step - loss: 0.4948 - mean_absolute_error: 0.4948
Epoch 324/500
13/13 [==============================] - 0s 3ms/step - loss: 0.5360 - mean_absolute_error: 0.5360
Epoch 325/500
13/13 [==============================] - 0s 2ms/step - loss: 0.4866 - mean_absolute_error: 0.4866
Epoch 326/500
13/13 [==============================] - 0s 3ms/step - loss: 0.5175 - mean_absolute_error: 0.5175
Epoch 327/500
13/13 [==============================] - 0s 2ms/step - loss: 0.5337 - mean_absolute_error: 0.5337
Epoch 328/500
13/13 [==============================] - 0s 2ms/step - loss: 0.5218 - mean_absolute_error: 0.5218
Epoch 329/500
13/13 [==============================] - 0s 3ms/step - loss: 0.4985 - mean_absolute_error: 0.4985
Epoch 330/500
13/13 [==============================] - 0s 2ms/step - loss: 0.5224 - mean_absolute_error: 0.5224
Epoch 331/500
13/13 [==============================] - 0s 3ms/step - loss: 0.5049 - mean_absolut

In [52]:

feature_list = ["1-rad", "inter-derivative", 
                    "bb-upper", "bb-lower", "bb-middle", 
                    "ema-cross", "sma-cross", 
                    "macdsignal", "macd", "macdhist",
                    "rsi", "sar-diff", 
                    "intra-derivative", "1-err", "intra-diff"]

test_features =  data[feature_list][:-1]
test_features = np.nan_to_num(test_features.values)
test_features = normalize(test_features)

test_labels = data["inter-diff"][1:]
m1.evaluate(test_features, test_labels, verbose=2)
# m2.evaluate(test_features, test_labels, verbose=2)

187/187 - 0s - loss: 1.9984 - mean_absolute_error: 1.9984


[1.998398780822754, 1.998398780822754]